In [1]:
import numpy as np
from probttrtypes import Type, PConstraint, BType, PType, Pred,\
Possibility, MeetType, JoinType, RecType, Fun, DisjProb, PDiv, PTimes, PExtreme, NegType
from ttrtypes import showmodel
from utils import show, show_latex, ttrace, nottrace, to_latex
from records import Rec

# Judging probabilities

The witness cache in `probttr` is a pair whose first member is a list of objects and whose second member is a list of probabilities (actually probability constraints)

In [2]:
T = Type()
print(T.witness_cache)

([], [])


In [3]:
show(T.judge('a',.5))

'0.5'

In [4]:
show(T.witness_cache)

'([a], [0.5])'

In [5]:
show(T.judge('a',.6))

'0.6'

In [6]:
show(T.witness_cache)

'([a], [0.6])'

Adding an additional probability argument to `judge()` gives you a minimum and maximum probability constraint.

In [7]:
show(T.judge('a',.6,1))

'>=0.6'

In [8]:
show_latex(T.judge('a',.6,.8))

<IPython.core.display.Latex object>

In [9]:
show_latex(T.judge('a',0,.6))

<IPython.core.display.Latex object>

`judge(a,n,n)` is the same as `judge(a,n)`

In [10]:
show_latex(T.judge('a',.6,.6))

<IPython.core.display.Latex object>

Probabilities must be between 0 and 1.  This can be checked by using the method `validate()` on a probability constraint.

In [11]:
print(T.judge('a',-1).validate())
print(T.judge('a',.6,.1).validate())

-1.0 is less than 0.
False
0.6 is greater than 0.1
False


In contrast to non-probabilistic `pyttr` we can store a negative judgement in witness cache.

In [12]:
T.judge('a',0)
show_latex(T.witness_cache)

<IPython.core.display.Latex object>

For compatibility with non-probabilistic TTR:  `judge(a)` is the same as `judge(a,1)` (which is the same as `judge(a,1,1)`).

In [13]:
show(T.judge('a'))

'1.0'

## Non-specific judging

We can also make judgements about the probability that something belongs to a type using the method `judge_nonspec`.

In [14]:
T_new = Type()
T_new.judge_nonspec(.3,.4)
show(T_new.prob_nonspec)

'>=0.3&<=0.4'

# Querying probabilities

## Querying unconditional probabilities

If `a` is in the witness cache, `query(a)` returns the probability stored in the witness cache for `a`.

In [15]:
show(T.query('a'))

'1.0'

If `a` is not in the witness cache and we have no other way of computing a probability that `a` is a witness for `T`, then the probability range is `[0,1]`, i.e. $\leq 1$.  This corresponds to returning an answer `Don't know` or `Undecided`.

In [16]:
show(T.query('b'))

'<=1.0'

`Don't know` results are not added to the witness cache.  (This may or may not be a good idea.)

In [17]:
show(T.witness_cache)

'([a], [1.0])'

Witness conditions are functions which return probability constraints. Here is type `Real` for real numbers, implemented as floating point decimals where the witness condition gives a categorical judgement for any object, that is it returns probability 1 or 0.

In [18]:
def RealClassifier(n):
    if isinstance(n,float):
        return PConstraint(1)
    else:
        return PConstraint(0)
Real = Type('Real')
Real.learn_witness_condition(RealClassifier)
show(Real.query(.6))

'1.0'

In [19]:
show(Real.query('a'))

'0.0'

In [20]:
show(Real.witness_cache)

'([0.6, a], [1.0, 0.0])'

Here's an example with a couple of witness conditions which return probability constraints.  `query(a)` returns the maximum obtained by the witness conditions for `a`.  Note that this need not be identical with either of the constraints returned by the individual witness conditions since the maximum of a collection of probability constraints is defined has as minimum the maximum of all the minima and as maximum the maximum of all the maxima.  (It could, of course, be done differently...)

In [21]:
T_at = Type('T_at')
def Classifier_a(s):
    if 'a' in s:
        return PConstraint(.8,.9)
    else:
        return PConstraint(.1,.3)
def Classifier_t(s):
    if 't' in s:
        return PConstraint(.2,.95)
    else:
        return PConstraint(.15,.7)
T_at.learn_witness_condition(Classifier_a)
T_at.learn_witness_condition(Classifier_t)
show(T_at.query('a'))

'>=0.8&<=0.9'

In [22]:
show(T_at.query('t'))

'>=0.2&<=0.95'

In [23]:
show(T_at.query('at'))

'>=0.8&<=0.95'

In [24]:
show(T_at.query('b'))

'>=0.15&<=0.7'

## Querying conditional probabilities

Conditions are provided as a second argument to the `query()` method as a list each of whose members is _either_ a tuple, `(a,T)`, where `a` is an object and `T` is a type, _or_ a type.  The idea is that, for example, `T.query(a,[(b,T1),T2])` queries the probability that `a` is of type `T` given that `b` is of type `T1` and there is some witness for `T2`.  

Consider a query `T.query(a,Conds)`.  If the probability that `a` is of type `T` does not depend on any of the conditions in `Conds` then what is returned is the same as `T.query(a)`, that is the unconditional probability. The default assumption is that probabilities are independent.

In [25]:
T1 = Type()
T2 = Type()
T1.judge('a',.6)
show(T1.query('a',[('b',T2)]))

'0.6'

One kind of dependency between probabilities relates to subtyping in the type theory.  Suppose that $T_2\sqsubseteq T_1$. Then $p(a:T_1|a:T_2)=1$. A limit case of this is where we have the same type judgement in the conditions as the one we are querying (types are subtypes of themselves): $p(a:T|a:T)$, that is, the probability that $a$ is of type $T$ given that $a$ is of type $T$ has to be $1$, no matter what the unconditional probability is that $a$ is of type $T$.

In [26]:
show(T1.query('a',[('a',T1)]))

'1.0'

Let us now create a type `T3` which is a subtype of `T1`

In [27]:
T3 = Type()
T1.learn_witness_condition(lambda x: T3.query(x))
T3.subtype_of(T1)

True

What is the probability that something is of type `T1` given that it is of type `T3`?

In [28]:
show(T1.query('b',[('b',T3)]))

'1.0'

Dependent probabilities that are not related to subtyping have to be provided by an oracle which is given as a third argument to `query()`.  An oracle is a python function which takes three arguments:  an object, a type and a list of conditions of the kind  which is used as an argument to `query()`. For any such argument it returns either a probability constraint (e.g. `PConstraint(.6)`) or `None`.  As a python function the oracle may call on resources external to `pyttr`, for example, conditional probability tables or Bayesian networks.  A call `T.query(a,c,o)` where `c` does not contain `(a,TT)` where `TT` is a subtype of `T` will return `o(a,T,c)` if this is not `None`. Otherwise `T.query(a,c,o)` will return `T.query(a)` (the unconditional probability).  If `c` _does_ contain `(a,TT)` where `TT` is a subtype of `T`, then the oracle will be ignored and `T.query(a,c,o)` will return `PConstraint(1)`.

As an example we define a trivial oracle.

In [29]:
def SillyOracle(a,T,c):
    if a == 'a'and T is T1 and ('b',T2) in c:
        return PConstraint(.7,.8)

Using the oracle.

In [30]:
show(T1.query('a',[('b',T2)],SillyOracle))

'>=0.7&<=0.8'

The oracle is not defined (returns `None`) and the result is the unconditional probability.

In [31]:
show(T1.query('a',[T2],SillyOracle))

'0.6'

The oracle is defined but is ignored because of the subtyping condition.

In [32]:
show(T1.query('a',[('b',T2),('a',T3)],SillyOracle))

'1.0'

The probability that $a$ is to the left of $b$ is the same as the probability the $b$ is to the right of $a$.

In [33]:
Ind = BType('Ind')
Ind.judge('a')
Ind.judge('b')
left = Pred('left',[Ind,Ind])
right = Pred('right',[Ind,Ind])
left.learn_witness_fun(lambda args: PType(right,[args[1],args[0]]))
right.learn_witness_fun(lambda args: PType(left,[args[1],args[0]]))
left_a_b = PType(left,['a','b'])
right_b_a = PType(right,['b','a'])
M = Possibility('M')
right_b_a.in_poss(M).judge('s1',.6)
left_a_b.in_poss(M).judge('s2',.7)
print(show(M))
print(show(left_a_b.in_poss(M).query('s1')))
print(show(right_b_a.in_poss(M).query('s2')))
print(show(M))


M:
_____________________________________________
right(b, a): [(s1, 0.6)]
left(a, b): [(s2, 0.7)]
_____________________________________________

0.6
0.7

M:
_____________________________________________
right(b, a): [(s1, 0.6), (s2, 0.7)]
left(a, b): [(s2, 0.7), (s1, 0.6)]
_____________________________________________



What happens if we revise our original judgement?  This shows the downside of caching the result of an inference in a dynamic environment where a premise for the inference has changed since we last checked.  Below we rejudge `left(a,b)` to have probability `.3` in `s2`.  However, when we query `right(b,a)` in `s2`, we still have the old value we found with the earlier judgement for `left(a,b)`.

In [34]:
left_a_b.in_poss(M).judge('s2',.3)
print(show(M))
print(show(right_b_a.in_poss(M).query('s2')))
print(show(M))


M:
_____________________________________________
right(b, a): [(s1, 0.6), (s2, 0.7)]
left(a, b): [(s2, 0.3), (s1, 0.6)]
_____________________________________________

0.7

M:
_____________________________________________
right(b, a): [(s1, 0.6), (s2, 0.7)]
left(a, b): [(s2, 0.3), (s1, 0.6)]
_____________________________________________



This is clearly the wrong result for the new circumstances.  A way around this is to `forget()` the probability for `s2` before calling `query()`.  Of course, knowing what needs to be forgotten is a delicate problem related to work on belief revision.  A good strategy is perhaps to always forget before querying when there is any possibility that something relevant might have changed.  `forget()` returns the old probability it found just in case you want to save it and reinstate it later if you are unable to compute a new probability.

In [35]:
print(show(right_b_a.in_poss(M).forget('s2')))
print(show(M))
print(show(right_b_a.in_poss(M).query('s2')))
print(show(M))

0.7

M:
_____________________________________________
right(b, a): [(s1, 0.6)]
left(a, b): [(s2, 0.3), (s1, 0.6)]
_____________________________________________

0.3

M:
_____________________________________________
right(b, a): [(s1, 0.6), (s2, 0.3)]
left(a, b): [(s2, 0.3), (s1, 0.6)]
_____________________________________________



## Non-specific querying

Non-specific queries are computed on the basis of a sample of objects which have been previously judged or queried in relation to the type (including objects which have been judged to have 0 probability of being a witness for the type).  The size of the sample is controlled by the variable `sample_size` in the file `config.py`.  A list representing a random sample of objects that have been judged for a type, `T`, is returned by `T.sample()`. If the number of objects that have been judged for `T` is less than `sample_size` then a list of all the objects which have been judged is returned.  We can also use `T.sample(n)` to return a sample of as many objects as have been judged up to `n`. 

In [36]:
print(show(Ind.sample()))
print(show(Ind.sample(1)))

[a, b]
[b]


The query `T.query_nonspec()` asks for the probability that there is something of type `T`, $p(T)$ in the notation of probabilistic TTR, though estimated on the basis of a random sample.  If something in the sample has been judged to be of type `T` with probability 1, then `T.query_nonspec()` returns probability 1.

In [37]:
show(Ind.query_nonspec())

'1.0'

Alternatively, if there is nothing in the sample but we have made a non-specific judgement using `judge_nonspec`, then the result of that non-specific judgement is returned.

In [38]:
show(T_new.query_nonspec())

'>=0.3&<=0.4'

If no non-specific judgement has been made, then what is returned is the disjunctive probability of all the probabilities for objects in the sample.

In [39]:
print(show(M))
print(show(right_b_a.in_poss(M).query_nonspec()))
print(show(left_a_b.in_poss(M).query_nonspec()))
print(show(M))


M:
_____________________________________________
right(b, a): [(s1, 0.6), (s2, 0.3)]
left(a, b): [(s2, 0.3), (s1, 0.6)]
_____________________________________________

0.72
0.72

M:
_____________________________________________
right(b, a): [(s1, 0.6), (s2, 0.3)]
left(a, b): [(s2, 0.3), (s1, 0.6)]
_____________________________________________



If a non-specific judgement has been made and there is a non-empty sample, then what is returned is the maximum of the non-specific judgement and the disjunctive probability of the probabilities for the objects in the sample.  Suppose, for example, that I make a judgement that there is .5 probability that there is something of type `Ind`, contrary to what is represented in the sample which shows objects with probability 1.  Then the result from the sample takes precedence.  If on the other hand I have made a judgement that there is certain likelihood of something being of a certain type and this exceeds the evidence represented in the sample, then the non-specific judgement is returned.

In [40]:
Ind.judge_nonspec(.5)
print(show(Ind.query_nonspec()))
T_new.judge('a',.1)
print(show(T_new.query_nonspec()))

1.0
>=0.3&<=0.4


Non-specific queries can also be conditional.  Here we show the probability that there is some situation in the type `right(b,a)` (i.e. that it is true that `b` is to the right of `a`) given that `s3` is a witness of `left(a,b)`.

In [41]:
print(show(right_b_a.in_poss(M).query_nonspec([('s3',left_a_b.in_poss(M))])))
print(show(M))

1.0

M:
_____________________________________________
right(b, a): [(s1, 0.6), (s2, 0.3)]
left(a, b): [(s2, 0.3), (s1, 0.6)]
_____________________________________________



Below we show the probability that there is some witness for the type `right(b,a)`, given that there is some witness for the type `left(a,b)`.

In [42]:
print(show(right_b_a.in_poss(M).query_nonspec([left_a_b.in_poss(M)])))
print(show(M))

1.0

M:
_____________________________________________
right(b, a): [(s1, 0.6), (s2, 0.3)]
left(a, b): [(s2, 0.3), (s1, 0.6)]
_____________________________________________



If the type in the conditions is not a subtype of the type being queried (in the case below because it refers to a different model), then what is returned is the unconditional probability, unless we provide a relevant oracle.

In [43]:
M1 = Possibility('M1')
print(show(right_b_a.in_poss(M).query_nonspec([left_a_b.in_poss(M1)])))

0.72


Recall that oracles are Python functions with three arguments: an object/event, `a`, a type `T` and a list of conditions, `c`.  If an oracle is to give us a result for a non-specific query, for example, the probability that there will be a `Rain` event given that there is an event of `DarkClouds`, then the oracle does not need the first argument.  Rather than having oracles with different numbers of arguments and making it difficult to have a single oracle which can provide conditional probabilities for both specific and non-specific queries, we will have the first argument set to `None` in the non-specific case.

In [44]:
DarkClouds = Type()
Rain = Type()
def OracleRain(a,T,c):
    if a is None and T is Rain and DarkClouds in c:
        return PConstraint(.85)
show(Rain.query_nonspec([DarkClouds],OracleRain))

'0.85'

### Double stroke conditional probabilities

The conditional probabilities represented above represent the probability that there is a situation of the type `right_b_a` given that there is a situation of the type `left_b_a`.  However, given the way we have characterized the relationship between `left` and `right` using the method `learn_witness_fun`, there is a stronger condition we could place on the probability:  we can talk of the probability for any situation, $s$, that $s$ is of the type `right_b_a` given that $s$ is of type `left_b_a`.  In the papers on probabilistic TTR we use $p(T_1||T_2)$ (with the double stroke) to represent the probability for any $s$ that $s:T_1$ given that $s:T_2$ and $p(T_1|T_2)$ (with a single stroke) to represent the probability that there is an $s$, $s:T_1$, given that there is an $s'$, $s':T_2$.  Note that $\exists s\ (s:T_1\wedge s:T_2)$ entails $\exists s\ s:T_1 \wedge \exists\ s' s':T_2$ and that for this reason $p(T_1||T_2)\leq p(T_1|T_2)$ in standard probability models (though, as we will see below, `pyttr` oracles will allow us to use non-standard models in which this does not hold).

In `pyttr` the double stroke condition is implemented by the method `query_doublecond` which requires a list of types as its first argument and also allows an optional second argument for an oracle.

In [45]:
show(right_b_a.in_poss(M).query_doublecond([left_a_b.in_poss(M)]))

'1.0'

Let us consider the difference between the two probabilities, $p(T_1|T_2)$ and $p(T_1||T_2)$.  Suppose that we have an experiment where an event of pushing a particular button leads to a reward (say, the appearance of a peanut or a piece of chocolate) 75% of the time.  That is, the probability of a reward event given a button-pushing event is .75: $p(\text{Reward}|\text{Button})=.75$.  However, pushing the button is not a reward in itself, that is, the probability of a button-pushing event being a reward event is 0: $p(\text{Reward}||\text{Button})=0$.  

Let us illustrate this with a small `pyttr` implementation of the example where we interpret $p(\text{Reward}|\text{Button})=.75$ as "the probability of the _next_ event after a button event being a reward event is .75" and we take a list of events to determine which events have been observed in order.

In [46]:
Button = Type()
Reward = Type()
events = ['e1','e2','e3','e4','e5','e6','e7']
for e in ['e1','e3','e5','e6']:
    Button.judge(e)
for e in ['e2','e4','e7']:
    Reward.judge(e)
def OracleButtonReward(eventlist):
    def Oracle(a,T,c):
        if T == Reward and (a,Button) in c:
            return PConstraint(0)
        elif T == Button and (a,Reward) in c:
            return PConstraint(0)
        elif a == None and T == Reward and Button in c:
            buttons = [e for e in events if Button.query(e).min==1]
            buttonswithrewards = [e for e in buttons 
                                  if Reward.query(events[events.index(e)+1]).min==1]
            return PConstraint(len(buttonswithrewards)/len(buttons))
    return Oracle
print(show(Reward.query_nonspec([Button],OracleButtonReward(events))))                    
print(show(Reward.query_doublecond([Button],OracleButtonReward(events)))) 

0.75
0.0


Let us now consider an example where $p(T_1||T_2)>0$.  Suppose that half of all green apples are tart as opposed to sweet, that is $p(\textit{TartApple}||\textit{GreenApple})=.5$.  The probability that there will be a tart apple given that there is a green apple will now be at least $.5$, that is, $p(\textit{TartApple}|\textit{GreenApple})\geq .5$. 

In [47]:
GreenApple = Type()
TartApple = Type()
apples = ['a1','a2','a3','a4','a5','a6','a7','a8','a9','a10']
for a in apples[:-2]:
    GreenApple.judge(a)
for a in apples[8:]:
    GreenApple.judge(a,0)
for a in apples[0:4]:
    TartApple.judge(a)
def OracleApple(applelist):
    def Oracle(a,T,c):
        if T == TartApple and (a,GreenApple) in c:
            greenapples = [a for a in applelist if GreenApple.query(a).min==1]
            tartgreenapples = [a for a in greenapples if TartApple.query(a).min==1]
            return PConstraint(len(tartgreenapples)/len(greenapples))
        elif a == None and T == TartApple and GreenApple in c:
            greenapples = DisjProb([(a,GreenApple) for a in applelist])
            tartapples = DisjProb([(a,TartApple) for a in applelist])
            return PDiv(PTimes(tartapples,greenapples),greenapples)
    return Oracle
print(show(TartApple.query('a11')))
print(show(TartApple.query('a11',[('a11',GreenApple)],OracleApple(apples))))
print(show(TartApple.query_nonspec([GreenApple],OracleApple(apples))))
print(show(TartApple.query_doublecond([GreenApple],OracleApple(apples))))

<=1.0
0.5
1.0
0.5


In the example above the first query shows that we don't know whether `a11` is a tart apple. The second query shows that it has 50% probability of being a tart apple if it is a green apple.  The third query shows the probability according to our data that there are tart apples and green apples divided by the probability that there are green apples. (Our data show that there are both green apples and tart apples.) Finally, the fourth query shows the probability that a given apple is tart given that it is green.  

In the examples above we have shown examples where the method `T1.query_doublecond(T2,oracle)` returns a unique probability.  They are examples where the oracle computes the same conditional probability `T1.query(a,[(a:T2)],oracle)` for each `a` in the data.  Now let us consider an example where this is not the case.  Suppose that we have two types $\textit{SeriouslyIll}$ and $\textit{HaveFlu}$.  Now we can ask the question: what is the probability that somebody will be seriously ill given that they have flu?  That is, what is $p(\textit{SeriouslyIll}||\textit{HaveFlu})$.  The probability for any particular individual depends on another factor:  the age of the person who has flu.  In probabilistic `pyttr` we will exploit the fact that we are using probability intervals to indicate the range of probabilities in a case like this where there is not unique probability shared by all elements in the data set.

In [48]:
HaveFlu = Type('HaveFlu')
SeriouslyIll = Type('SeriouslyIll')
Over80 = Type()
Between50and79 = Type()
Between2and49 = Type()
Under2 = Type()
patients = ['a1','a2','a3','a4','a5','a6','a7','a8','a9','a10']
for a in patients:
    HaveFlu.judge(a)
Over80.judge('a10')
for a in patients[5:9]:
    Between50and79.judge(a)
for a in patients[1:5]:
    Between2and49.judge(a)
Under2.judge('a1')
def OracleRiskSeriouslyIll(a,T,c):
    if T is SeriouslyIll and (a,HaveFlu) in c:
        if Over80.query(a).min == 1:
            return PConstraint(.6)
        elif Between50and79.query(a).min == 1:
            return PConstraint(.3)
        elif Between2and49.query(a).min == 1:
            return PConstraint(.05)
        elif Under2.query(a).min == 1:
            return PConstraint(.1)
    elif a is None and T is SeriouslyIll and HaveFlu in c:
        return PConstraint(T.query_doublecond(c,OracleRiskSeriouslyIll).min,1)
print(show(SeriouslyIll.query_doublecond([HaveFlu],OracleRiskSeriouslyIll)))
print(show(SeriouslyIll.query_nonspec([HaveFlu],OracleRiskSeriouslyIll)))

        

>=0.05&<=0.6
>=0.05


The results above show that in our example the probability that somebody will be seriously ill given that they have flu, $p(\textit{SeriouslyIll}||\textit{Flu})$, ranges between .05 and .6 whereas the likelihood that somebody will be seriously ill, given that somebody has flu, $p(SeriouslyIll|Flu)$ is at least .05. Note that in the definition of the oracle above, it is the last conditional clause in the definition which requires the relationship between the two probabilities.  If this clause is removed then the final result for the single stroke probability would be [0,1], that is, "Don't know", which is not greater than the double stroke probability and thus the oracle would define a non-standard model in the terms we introduced above.

## Meet types

If we judge that the probability of `a` being of type `MeetType(T1,T2)` is `1`, then we also judge the probability of `a` being of `T1` and the probability of `a` being of `T2` to be `1`.

In [49]:
Tleft = Type()
Tright = Type()
Tm = MeetType(Tleft,Tright)
Tm.judge('a')
print(show(Tleft.query('a')))
print(show(Tright.query('a')))

1.0
1.0


Otherwise, we do not currently draw any conclusions about the probabilities for the component types.

In [50]:
Tleft1 = Type()
Tright1 = Type()
Tm1 = MeetType(Tleft1,Tright1)
Tm1.judge('a',.6,.8)
print(show(Tleft1.query('a')))
print(show(Tright1.query('a')))
print(show(Tm1.query('a')))

<=1.0
<=1.0
>=0.6&<=0.8


The user may wish to decide that the `judge` method is not to be used with meet types, only `query`, that is, that the `judge` method is restricted to basic types.  However, making judgements about join types may be useful.  See below.

Similar remarks hold for non-specific judgements.

In [51]:
Tleft2 = Type()
Tright2 = Type()
Tm2 = MeetType(Tleft2,Tright2)
Tm2.judge_nonspec()
print(show(Tleft2.query_nonspec()))
print(show(Tright2.query_nonspec()))
Tleft3 = Type()
Tright3 = Type()
Tm3 = MeetType(Tleft3,Tright3)
Tm3.judge_nonspec(.6,.8)
print(show(Tleft3.query_nonspec()))
print(show(Tright3.query_nonspec()))
print(show(Tm3.query_nonspec()))

1.0
1.0
<=1.0
<=1.0
>=0.6&<=0.8


If an object is not in the witness cache of a meet type then the conjunctive probability of the values returned for the two components is returned by the `query()` method.

In [52]:
Tleft3.judge('a',.6)
print(show(Tright3.query('a')))
show(Tm3.query('a'))

<=1.0


'<=0.6'

If an object is in the witness cache then the probability stored there will be returned, even though there may be conflicting evidence in the two components.  In order to get the new value we need to `forget()`

In [53]:
Tright3.judge('a',.3)
print(show(Tm3.query('a')))
Tm3.forget('a')
print(show(Tm3.query('a')))

<=0.6
0.18


The computation of conjunctive probability uses an adaptation of the Kolmogorov formula for conjunction: $p(a:T_1\wedge T_2) = p(a:T_1)p(a:T_2\mid a:T_1)$, as given in Cooper et al. (2015).  This involves a conditional probability and therefore in the implementation and oracle argument creating a dependence between the two types will make a difference to the outcome when querying a meet type.

In [54]:
def Oracle1(a,T,c):
    if a == 'a'and T is Tright3 and ('a',Tleft3) in c:
        return PConstraint(.7,.8)
def Oracle2(a,T,c):
    if a == 'a'and T is Tright3 and ('a',Tleft3) in c:
        return PConstraint(0)
Tm3.forget('a')
print(show(Tm3.query('a',oracle=Oracle1)))
Tm3.forget('a')
print(show(Tm3.query('a',oracle=Oracle2)))

>=0.42&<=0.48
0.0


Conditional probabilities can also be queried for meet types.  

In [55]:
Tm3.forget('a')
print(show(Tm3.query('a',[('a',Tleft3),('a',Tright3)])))

1.0


The above example shows the need for witness conditions which pass the conditions and oracle arguments to components of the type. The witness condition for meet types is, schematically, 
`lambda a,c,oracle: ConjProb([(a,<left>),(a,<right>)],c,oracle)`.  (As in non-probabilistic TTR, meet types and other "logical types" cannot learn new witness conditions.)  Note that this witness condition has three arguments so that it can pass the conditions, `c`, and the oracle to the function `ConjProb` which computes the conjunctive probability.  Witness conditions in the `probttr` implementation can have one to three arguments and will be applied to the arguments provided to the `query` method from which they are called:  the object, `a`, being queried for one argument, `a` and the conditions, `c`, for two arguments and `a`, `c` and the oracle for three arguments.

## Join types

Join types work like meet types _mutatis mutandis_.  Schematically, the witness condition is `lambda a,c,oracle: DisjProb([(a,<left>),(a,<right>)],c,oracle)` and no new witness conditions can be learnt.  If we judge something to have 0 probability of being of a join type, then we judge it to have 0 probability of being of the two component types.

In [56]:
Tleftd = Type()
Trightd = Type()
Tmd = JoinType(Tleftd,Trightd)
Tmd.judge('a',0)
print(show(Tleftd.query('a')))
print(show(Trightd.query('a')))

0.0
0.0


Similarly for non-specific judgements.

In [57]:
Tleft2d = Type()
Tright2d = Type()
Tm2d = JoinType(Tleft2d,Tright2d)
Tm2d.judge_nonspec(0)
print(show(Tleft2d.query_nonspec()))
print(show(Tright2d.query_nonspec()))
Tleft3d = Type()
Tright3d = Type()
Tm3d = JoinType(Tleft3,Tright3)
Tm3d.judge_nonspec(.6,.8)
print(show(Tleft3d.query_nonspec()))
print(show(Tright3d.query_nonspec()))
print(show(Tm3d.query_nonspec()))

0.0
0.0
<=1.0
<=1.0
>=0.6&<=0.8


## Record types

The probability that a record, $r$, is of a record type, $T$, is the conjunctive probability of the probabilities that the objects in the fields of $r$ are of the types in the correspondingly labelled types of $T$.  If there is a label in $T$ which is not in $r$ then the probability that $r$ is of type $T$ is 0 (also if $r$ is not a record at all then the probability is 0).

A simple non-dependent record type.

In [58]:
Tf1 = Type()
Tf2 = Type()
Tr1 = RecType({'l1':Tf1,'l2':Tf2})
Tf1.judge('a',.3)
Tf2.judge('b',.2)
r1 = Rec({'l1':'a','l2':'b'})
print(show(Tr1.query(r1)))

0.06


A simple dependent record type.

In [59]:
dog = Pred('dog',[Ind])
a_dog = RecType({'x':Ind,'e':(Fun('v', Ind, PType(dog,['v'])),['x'])})
show_latex(a_dog)

<IPython.core.display.Latex object>

In [60]:
Ind.judge('d')
PType(dog,['d']).judge('s1',.7)
r2 = Rec({'x':'d','e':'s1','z':'other_stuff'})
show(a_dog.query(r2))

'0.7'

A simple record type with a path leading to another record type

In [61]:
Tr2 = RecType({'x':a_dog})
show_latex(Tr2)

<IPython.core.display.Latex object>

In [62]:
r3 = Rec({'x':r2})
show(Tr2.query(r3))

'0.7'

A slightly more complex record type.

In [63]:
bark = Pred('bark',[Ind])
a_dog_bark = RecType({'x':a_dog,'e':(Fun('v',Ind,PType(bark,['v'])),['x.x'])})
show_latex(a_dog_bark)

<IPython.core.display.Latex object>

In [64]:
PType(bark,['d']).judge('s2',.3)
r4 = Rec({'x':r2,'e':'s2'})
show_latex(r4)

<IPython.core.display.Latex object>

In [65]:
show(a_dog_bark.query(r4))

'0.21'

Conditional probabilities.

In [66]:
show(a_dog_bark.query(r4,[('s2',PType(bark,['d']))]))

'0.7'

In [67]:
show(a_dog_bark.query(r4,[('s1',PType(dog,['d'])),('s2',PType(bark,['d']))]))

'1.0'

Non-specific queries.

Samples for record types are based not on records that have previously been judge for that type but on judgements that have been made for the types within the record type.  Suppose, for example, that we create a new record type which we have not previously judged but which contains types for which there have been judgements.

In [68]:
adb = RecType({'x':Ind,'c':(Fun('v', Ind, PType(dog,['v'])),['x']),'e':(Fun('v', Ind, PType(bark,['v'])),['x'])})
show_latex(adb)               

<IPython.core.display.Latex object>

We should be able to sample witnesses for this type given that we have a witness for individuals which are dogs and bark, even though we have not judged or queried this particular type.

In [69]:
print(show(adb.sample()))

[]


This means that we are able to compute the probability that there is a record of the type `adb` by computing the disjunctive probability of the elements in the sample belonging to the type in the usual way.

In [70]:
show(adb.query_nonspec())

'<=1.0'

Given that the record types `adb` and `a_dog_bark` contain the same basic information, although structured differently the probability that they have a witness will be the same (or approximately the same given that the random samples chosen for the two types might be different in larger examples) regardless of the history of judgements that have been made for those particular record types.

In [71]:
show(a_dog_bark.query_nonspec())

'<=1.0'

## Negation of types

The negation of a type is treated as an object of a type class.  `NegType(T)` creates a type which is the negation of `T`.  Probabilities associated with the negation of a type, `T` are computed in terms of `1-p` where `p` is the probability associated with `T`.  For example, `NegType(T).query(a)` has the value computed by: 

>`PConstraint(1-T.query(a).max,1-T.query(a).min)`

Below we show an example using the `query_nonspec` method.

In [72]:
show(NegType(a_dog_bark).query_nonspec())

'<=1.0'